In [1]:
from nipype import Node, Workflow, IdentityInterface, MapNode,SelectFiles, DataSink
from nipype.interfaces import fsl
import numpy as np
import os

In [2]:
data = "/home/paradeisios/Downloads/flanker"
output = "/home/paradeisios/Desktop/nipype_output/preprocessing"

num_of_subjects = 1
num_of_runs = 2

subject_list = ["{:02d}".format(i) for i in range(1,num_of_subjects+1)]

mni_template = "/home/paradeisios/fsl/data/standard/MNI152_T1_2mm.nii.gz"
mni_brain_template = "/home/paradeisios/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz"

TR = 2.5

In [3]:
infosource = Node(IdentityInterface(fields=["subject_list"]), name="infosource")
infosource.iterables = [('subject_list', subject_list)]

In [4]:
templates = {"anat": "sub-{subject_list}/anat/sub-{subject_list}_T1w.nii.gz",
             "func": "sub-{subject_list}/func/sub-{subject_list}_task-flanker_run-*_bold.nii.gz",
             "conditions": "sub-{subject_list}/func/sub-{subject_list}_task-flanker_run-*_events.tsv"}

selectfiles = Node(SelectFiles(templates),name="selectfiles")
selectfiles.inputs.base_directory = data


substitutions = [("_mean_img","mean_img"),
                 ("_subject_list",""),
                 ("_flirt_func","flirt_func"),
                 ("_func_warp","func_warp")]

substitutions += [("_{:02d}".format(i), "{:02d}".format(i)) for i in range(1,num_of_subjects+1)]
substitutions += [("_art{:d}".format(i), "art_run_{:d}".format(i+1)) for i in range(num_of_runs)]
substitutions += [("mean_img{:d}".format(i), "mean_img_run_{:d}".format(i+1)) for i in range(num_of_runs)]
substitutions += [("_realign{:d}".format(i), "realign_{:d}".format(i+1)) for i in range(num_of_runs)]
substitutions += [("flirt_func{:d}".format(i), "flirt_func_run_{:d}".format(i+1)) for i in range(num_of_runs)]
substitutions += [("func_warp{:d}".format(i), "func_warp_run_{:d}".format(i+1)) for i in range(num_of_runs)]
substitutions += [("_smooth{:d}".format(i), "smooth_run_{:d}".format(i+1)) for i in range(num_of_runs)]
substitutions += [("_vol_removal{:d}".format(i), "vol_removal_{:d}".format(i+1)) for i in range(num_of_runs)]

substitutions += [("seg_0", "GM")]
substitutions += [("seg_1", "WM")]
substitutions += [("seg_2", "CSF")]

datasink = Node(DataSink(base_directory=output),name="datasink")
datasink.inputs.substitutions = substitutions
datasink.output = output

In [5]:
# structural workflow 

bet = Node(fsl.BET(),name="bet")
bet.inputs.frac = 0.3 #value controlling how much of the skull to be removed - increasing might remove bits of brain
bet.inputs.vertical_gradient = 0.2 #value to linearly improve estimations at the bottom of the brain -increasing might understimate top
bet.inputs.mask = True #creates a binary mask of the brain


segment = Node(fsl.FAST(),name="segment")
"""You are interested mainly in the restored version of the structural and
    pve0 = grey matter
    pve1 = white matter
    pve2 = CSF """
segment.inputs.img_type = 1 # indicate its a T1 image
segment.inputs.number_classes = 3 # seperate structural into Grey Matter, White Matter and CSF
segment.inputs.output_biascorrected = True #return structural image bias field corrected
segment.inputs.output_biasfield = True #return bias field
segment.inputs.segments=True #create a binary mask for each tissue type
segment.inputs.no_pve=False #if you dont want partial volume estimation, set to true

flirt_struct = Node(fsl.FLIRT(),name="flirt_struct")
"""This step performs linear registration of the structural volume to the standard space"""
flirt_struct.inputs.reference = mni_brain_template
flirt_struct.inputs.dof = 12 #use 12 dof in structural flirt to prepare image for FNIRT nonlinear normalization to mni space
flirt_struct.inputs.out_matrix_file = "h2s_affine.mat" #keep this to facilite struct and func normalization to mni space using FNIRT
flirt_struct.inputs.interp = "spline" # change to trilinear if you are mostly interested in non-subcortical normalization. Trilinear is faster, but might have some rotational issues
flirt_struct.inputs.cost = "mutualinfo"


fnirt_struct = Node(fsl.FNIRT(),name="fnirt_struct")
"""This step performs non-linear registration of the structural volume to the standard space"""
fnirt_struct.inputs.ref_file = mni_template
fnirt_struct.inputs.config_file = "T1_2_MNI152_2mm"
fnirt_struct.inputs.fieldcoeff_file = True  #keep


In [6]:
# functional workflow

img_to_float = MapNode(fsl.ImageMaths(),name="img_to_float",iterfield=['in_file'])
"""This converts the 4d in float representation to faciliate some further computations"""
img_to_float.inputs.out_data_type='float'
img_to_float.inputs.op_string=''


vol_removal = MapNode(fsl.ExtractROI(), name="vol_removal",iterfield=['in_file'])
"""This step removes the 5 first volumes to avoid T1 saturation"""
vol_removal.inputs.t_min = 0 #number of scans to exclude from functional image
vol_removal.inputs.t_size = -1 # keep all the rest until the end


realign = MapNode(fsl.MCFLIRT(),name = "realign",iterfield=['in_file'])
"""This step realigns data to the first image"""
realign.inputs.save_mats = True #save realignment  parameters
realign.inputs.save_plots = True #save parameter plots
realign.inputs.dof = 6 #use this for rigid body correction


mean_img = MapNode(fsl.maths.MeanImage(),name= "mean_img",iterfield=['in_file'])
mean_img.inputs.dimension = "T" # find mean image across the time dimension


flirt_func = MapNode(fsl.FLIRT(),name = "flirt_func",iterfield=['in_file'])
"""This step performs linear registration of the functional volume to the structural volume. It is mainly used to
extract the transformation matrix to later register to standard space"""
flirt_func.inputs.dof = 6
flirt_func.inputs.out_matrix_file = "f2h_affine.mat"
flirt_func.inputs.interp = "spline"
flirt_func.inputs.cost = "mutualinfo"


func_warp = MapNode(fsl.ApplyWarp(),name = "func_warp",
                    iterfield=['in_file',"premat"])
"""This step performs non linear registration to standard space using the affine matrix extracted
in the flirt above and the fnirt non linear warp in the structural pipeline """
func_warp.inputs.ref_file = mni_template


smooth = MapNode(fsl.IsotropicSmooth(),name="smooth",iterfield=['in_file'])
"""This step performs smoothing"""
smooth.inputs.fwhm = 6 #change the smoothing kernel

In [7]:
preprocessing = Workflow(name="preprocessing")

preprocessing.connect(infosource,"subject_list",selectfiles, "subject_list")
preprocessing.connect(infosource,"subject_list",datasink, "container")                  
preprocessing.connect(selectfiles,"anat",bet, "in_file")  
preprocessing.connect(selectfiles,"anat",fnirt_struct, "in_file") 
                 
preprocessing.connect(bet,"out_file",segment,"in_files")
preprocessing.connect(segment,"restored_image",flirt_struct,"in_file")
preprocessing.connect(flirt_struct,"out_matrix_file",fnirt_struct,"affine_file")

preprocessing.connect(selectfiles,"func",img_to_float, "in_file")   
preprocessing.connect(img_to_float,"out_file",vol_removal,"in_file")
preprocessing.connect(vol_removal, "roi_file", realign, "in_file")

preprocessing.connect(mean_img,"out_file",flirt_func,"in_file")
preprocessing.connect(realign,"out_file",func_warp,"in_file")
preprocessing.connect(flirt_func,"out_matrix_file",func_warp,"premat")
preprocessing.connect(segment,"restored_image",flirt_func,"reference")
preprocessing.connect(fnirt_struct,"fieldcoeff_file",func_warp,"field_file")
preprocessing.connect(func_warp,"out_file",smooth,"in_file")

In [8]:
preprocessing.write_graph(graph2use='flat',format='png', simple_form=True,
                          dotfilename= os.path.join(output,"preprocessing_workflow.dot"))

210805-17:25:37,49 nipype.workflow INFO:
	 Generated workflow graph: /home/paradeisios/Desktop/nipype_output/preprocessing/preprocessing_workflow.png (graph2use=flat, simple_form=True).


'/home/paradeisios/Desktop/nipype_output/preprocessing/preprocessing_workflow.png'

In [9]:
preprocessing.connect(selectfiles,"conditions",datasink,"conditions")
preprocessing.connect(bet,"out_file",datasink,"bet.@bet_brain")
preprocessing.connect(bet,"mask_file",datasink,"bet.@bet_mask")
preprocessing.connect(segment,"restored_image",datasink,"segment.@restored_image")
preprocessing.connect(segment,"tissue_class_files",datasink,"segment.@tissue_class_files")
preprocessing.connect(segment,"bias_field",datasink,"segment.@bias_field")
preprocessing.connect(segment,"tissue_class_map",datasink,"segment.@tissue_class_map")
preprocessing.connect(flirt_struct,"out_file",datasink,"flirt_struct.@structural_flirt")
preprocessing.connect(flirt_struct,"out_matrix_file",datasink,"flirt_struct.@affine_parameters")
preprocessing.connect(fnirt_struct,"warped_file",datasink,"flirt_struct.@structural_fnirt")
preprocessing.connect(fnirt_struct,"field_file",datasink,"flirt_struct.@fieldcoeff_file")
preprocessing.connect(vol_removal,"roi_file",datasink,"vol_removal.@files")
preprocessing.connect(realign,"out_file",datasink,"realign.@realigned_files")
preprocessing.connect(realign,"par_file",datasink,"realign.@realig_parameters")

preprocessing.connect(mean_img,"out_file",datasink,"mean_img.@mean_img")
preprocessing.connect(flirt_func,"out_matrix_file",datasink,"flirt_func.@affine_parameters")
preprocessing.connect(flirt_func,"out_file",datasink,"flirt_func.@functional_flirt")
preprocessing.connect(func_warp,"out_file",datasink,"func_warp.@functional_fnirt")
preprocessing.connect(smooth,"out_file",datasink,"smooth.@smoothed_files")

In [10]:
preprocessing.run(plugin='MultiProc', plugin_args={'n_procs' : 4})

210805-17:25:37,136 nipype.workflow INFO:
	 Workflow preprocessing settings: ['check', 'execution', 'logging', 'monitoring']
210805-17:25:37,169 nipype.workflow INFO:
	 Running in parallel.
210805-17:25:37,177 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 6.74/6.74, Free processors: 4/4.
210805-17:25:37,281 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing.selectfiles" in "/tmp/tmpczf_2c09/preprocessing/_subject_list_01/selectfiles".
210805-17:25:37,289 nipype.workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
210805-17:25:37,299 nipype.workflow INFO:
	 [Node] Finished "preprocessing.selectfiles".
210805-17:25:39,181 nipype.workflow INFO:
	 [Job 0] Completed (preprocessing.selectfiles).
210805-17:25:39,188 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 jobs ready. Free memory (GB): 6.74/6.74, Free processors: 4/4.
210805-17:25:39,261 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing.bet

210805-17:26:05,286 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing.vol_removal" in "/tmp/tmph0bw0f3l/preprocessing/_subject_list_01/vol_removal".
210805-17:26:05,292 nipype.workflow INFO:
	 [Node] Setting-up "_vol_removal0" in "/tmp/tmph0bw0f3l/preprocessing/_subject_list_01/vol_removal/mapflow/_vol_removal0".
210805-17:26:05,294 nipype.workflow INFO:
	 [Node] Cached "_vol_removal0" - collecting precomputed outputs
210805-17:26:05,296 nipype.workflow INFO:
	 [Node] "_vol_removal0" found cached.
210805-17:26:05,299 nipype.workflow INFO:
	 [Node] Setting-up "_vol_removal1" in "/tmp/tmph0bw0f3l/preprocessing/_subject_list_01/vol_removal/mapflow/_vol_removal1".
210805-17:26:05,302 nipype.workflow INFO:
	 [Node] Cached "_vol_removal1" - collecting precomputed outputs
210805-17:26:05,304 nipype.workflow INFO:
	 [Node] "_vol_removal1" found cached.
210805-17:26:05,427 nipype.workflow INFO:
	 [Node] Finished "preprocessing.vol_removal".
210805-17:26:07,203 nipype.workflow INFO:
	 [Jo

210805-17:27:11,269 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 0 jobs ready. Free memory (GB): 6.54/6.74, Free processors: 3/4.
                     Currently running:
                       * preprocessing.segment
210805-17:30:17,200 nipype.workflow INFO:
	 [Node] Finished "preprocessing.segment".
210805-17:30:17,447 nipype.workflow INFO:
	 [Job 6] Completed (preprocessing.segment).
210805-17:30:17,452 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 jobs ready. Free memory (GB): 6.74/6.74, Free processors: 4/4.
210805-17:30:17,509 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing.flirt_struct" in "/tmp/tmpmkao07k1/preprocessing/_subject_list_01/flirt_struct".
210805-17:30:17,534 nipype.workflow INFO:
	 [Node] Running "flirt_struct" ("nipype.interfaces.fsl.preprocess.FLIRT"), a CommandLine Interface with command:
flirt -in /tmp/tmpjbakwp7f/preprocessing/_subject_list_01/segment/sub-01_T1w_brain_restore.nii.gz -ref /home/paradeisios/fsl/data/standard/MNI1

210805-17:41:16,87 nipype.workflow INFO:
	 [MultiProc] Running 2 tasks, and 0 jobs ready. Free memory (GB): 6.34/6.74, Free processors: 2/4.
                     Currently running:
                       * _func_warp1
                       * _func_warp0
210805-17:43:25,127 nipype.workflow INFO:
	 [Node] Finished "_func_warp0".
210805-17:43:25,523 nipype.workflow INFO:
	 [Node] Finished "_func_warp1".
210805-17:43:26,223 nipype.workflow INFO:
	 [Job 23] Completed (_func_warp0).
210805-17:43:26,224 nipype.workflow INFO:
	 [Job 24] Completed (_func_warp1).
210805-17:43:26,226 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 6.74/6.74, Free processors: 4/4.
210805-17:43:26,281 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing.func_warp" in "/tmp/tmpjw_n2xfz/preprocessing/_subject_list_01/func_warp".
210805-17:43:26,289 nipype.workflow INFO:
	 [Node] Setting-up "_func_warp0" in "/tmp/tmpjw_n2xfz/preprocessing/_subject_list_01/func_warp/mapflow

210805-17:45:06,518 nipype.interface INFO:
	 sub: /home/paradeisios/Desktop/nipype_output/preprocessing/01/vol_removal/_subject_list_01/_vol_removal1/sub-01_task-flanker_run-2_bold_maths_roi.nii.gz -> /home/paradeisios/Desktop/nipype_output/preprocessing/01/vol_removal/01/vol_removal_2/sub-01_task-flanker_run-2_bold_maths_roi.nii.gz
210805-17:45:06,523 nipype.interface INFO:
	 sub: /home/paradeisios/Desktop/nipype_output/preprocessing/01/realign/_subject_list_01/_realign0/sub-01_task-flanker_run-1_bold_maths_roi_mcf.nii.gz -> /home/paradeisios/Desktop/nipype_output/preprocessing/01/realign/01/realign_1/sub-01_task-flanker_run-1_bold_maths_roi_mcf.nii.gz
210805-17:45:06,531 nipype.interface INFO:
	 sub: /home/paradeisios/Desktop/nipype_output/preprocessing/01/realign/_subject_list_01/_realign1/sub-01_task-flanker_run-2_bold_maths_roi_mcf.nii.gz -> /home/paradeisios/Desktop/nipype_output/preprocessing/01/realign/01/realign_2/sub-01_task-flanker_run-2_bold_maths_roi_mcf.nii.gz
210805-17:4